In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

# splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 50,)

splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 600,
    chunk_overlap = 100,
)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size = 600,
    chunk_overlap = 100,
)

loader = UnstructuredFileLoader("./files/1984_chap_one.docx")

# docs = loader.load()
# splitter.split_documents(docs)

result = loader.load_and_split(text_splitter=splitter )
print(len(result))

/var/folders/b1/wt2bjx71325fk62cxg92dwl80000gn/T/ipykernel_63505/3547876302.py:20: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the langchain-unstructured package and should be used instead. To use it run `pip install -U langchain-unstructured` and import as `from langchain_unstructured import UnstructuredLoader`.
  loader = UnstructuredFileLoader("./files/1984_chap_one.docx")


18


In [2]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_documents(["Hi", "how", "are", "you longer sentence because"])

print(len(vector), len(vector[0]))

/var/folders/b1/wt2bjx71325fk62cxg92dwl80000gn/T/ipykernel_63505/2431765116.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedder = OpenAIEmbeddings()


4 1536


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size = 600,
    chunk_overlap = 100,
)

loader = UnstructuredFileLoader("./files/1984_chap_one.docx")

doc = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(doc, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)

# chain.run("Where does Winston live?")
chain.run("Describe Victory Mansions")

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-DxyR7ijEF9ww6YcVJ24D6t40 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-DxyR7ijEF9ww6YcVJ24D6t40 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.

In [ ]:
result = vectorstore.similarity_search("where does winston live?")

print(len(result))
print(result)